In [ ]:
import pandas as pd
import itertools
from collections import defaultdict
import time
from mlxtend.frequent_patterns import apriori as apriori_lib
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

# Function to read transactions from a CSV file
def read_transactions_from_csv(file_path):
    data_frame = pd.read_csv(file_path, header=None)
    return [set(str(item) for item in transaction if pd.notna(item)) for transaction in data_frame.values.tolist()]

# Function to ensure valid float input for support and confidence
def get_valid_float(prompt, min_value, max_value):
    while True:
        try:
            user_input = float(input(prompt))
            if min_value <= user_input <= max_value:
                return user_input / 100  # Convert percentage to decimal
            else:
                print(f"Please provide a value between {min_value} and {max_value}.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

# Function to ensure valid integer input for store selection
def get_valid_int(prompt, min_value, max_value):
    while True:
        try:
            user_input = int(input(prompt))
            if min_value <= user_input <= max_value:
                return user_input
            else:
                print(f"Please provide a number between {min_value} and {max_value}.")
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

# Dictionary of stores and their corresponding transaction files
store_files = {
    "Amazon": "amazon_transactions.csv",
    "Best Buy": "bestbuy_transactions.csv",
    "Nike": "nike_transactions.csv",
    "K Mart": "k_mart_transactions.csv",
    "Target": "target_transactions.csv"
}

# Brute force method for finding frequent itemsets and generating rules
def brute_force_algorithm(transactions, min_support, min_confidence):
    def count_itemsets(itemsets):
        counts = defaultdict(int)
        for transaction in transactions:
            for itemset in itemsets:
                if set(itemset).issubset(transaction):
                    counts[itemset] += 1
        return counts

    unique_items = set(item for transaction in transactions for item in transaction)
    total_transactions = len(transactions)
    frequent_itemsets = {}
    k = 1

    while True:
        itemsets = list(itertools.combinations(unique_items, k))
        item_counts = count_itemsets(itemsets)
        frequent_items = {frozenset(item): count / total_transactions for item, count in item_counts.items() if count / total_transactions >= min_support}
        if not frequent_items:
            break
        frequent_itemsets[k] = frequent_items
        k += 1

    generated_rules = []
    for k in range(2, len(frequent_itemsets) + 1):
        for itemset in frequent_itemsets[k]:
            for i in range(1, k):
                for antecedent in itertools.combinations(itemset, i):
                    antecedent_set = frozenset(antecedent)
                    consequent_set = frozenset(itemset) - antecedent_set
                    if antecedent_set in frequent_itemsets[len(antecedent_set)]:
                        support = frequent_itemsets[k][itemset]
                        confidence = support / frequent_itemsets[len(antecedent_set)][antecedent_set]
                        if confidence >= min_confidence:
                            generated_rules.append((antecedent_set, consequent_set, confidence, support))

    return frequent_itemsets, generated_rules

# Apriori algorithm using the mlxtend library
def apriori_algorithm(transactions, min_support, min_confidence):
    encoder = TransactionEncoder()
    encoded_array = encoder.fit(transactions).transform(transactions)
    df = pd.DataFrame(encoded_array, columns=encoder.columns_)
    frequent_itemsets = apriori_lib(df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    return frequent_itemsets, rules

# Function to compare results from both algorithms
def compare_algorithms(brute_force_results, apriori_results):
    bf_itemsets, bf_rules = brute_force_results
    ap_itemsets, ap_rules = apriori_results

    print("\nResults Comparison:")
    print(f"Brute Force: {sum(len(itemsets) for itemsets in bf_itemsets.values())} frequent itemsets, {len(bf_rules)} rules")
    print(f"Apriori: {len(ap_itemsets)} frequent itemsets, {len(ap_rules)} rules")

    # Check if both methods yield the same results
    same_itemsets_count = (sum(len(itemsets) for itemsets in bf_itemsets.values()) == len(ap_itemsets))
    same_rules_count = len(bf_rules) == len(ap_rules)
    print(f"Same number of frequent itemsets: {same_itemsets_count}")
    print(f"Same number of rules: {same_rules_count}")

# Function to display the generated association rules
def show_rules(rules, algorithm_name):
    print(f"\n{algorithm_name} Association Rules:")
    if isinstance(rules, list):  # For brute force results
        for index, (antecedent, consequent, confidence, support) in enumerate(rules, 1):
            print(f"Rule {index}: {set(antecedent)} -> {set(consequent)} "
                  f"(Confidence: {confidence:.2f}, Support: {support:.2f})")
    elif isinstance(rules, pd.DataFrame):  # For library results (Apriori)
        for index, rule in rules.iterrows():
            antecedents = list(rule['antecedents'])
            consequents = list(rule['consequents'])
            print(f"Rule {index + 1}: {set(antecedents)} -> {set(consequents)} "
                  f"(Confidence: {rule['confidence']:.2f}, Support: {rule['support']:.2f})")
    else:
        print("No rules found or unsupported rule format.")

# Main loop for user interaction
while True:
    print("\nWelcome! Please select a store to analyze:")
    for idx, store in enumerate(store_files.keys(), 1):
        print(f"{idx}. {store}")
    
    # Store selection using integer input validation
    selected_store = get_valid_int("Enter the number of the store you want to analyze (1-5): ", 1, 5)
    store_name = list(store_files.keys())[selected_store - 1]
    
    try:
        # Load transactions from the selected store's CSV file
        transactions = read_transactions_from_csv(store_files[store_name])
        
        print(f"\nAnalyzing transactions for {store_name}:")
        min_support = get_valid_float("Enter minimum support (1-100): ", 1, 100)
        min_confidence = get_valid_float("Enter minimum confidence (1-100): ", 1, 100)
        
        print("\nExecuting algorithms...")
        
        # Run brute force algorithm
        start_time = time.time()
        bf_itemsets, bf_rules = brute_force_algorithm(transactions, min_support, min_confidence)
        bf_duration = time.time() - start_time
        
        # Run Apriori algorithm
        start_time = time.time()
        ap_itemsets, ap_rules = apriori_algorithm(transactions, min_support, min_confidence)
        ap_duration = time.time() - start_time
        
        print(f"\nBrute Force Execution Time: {bf_duration:.4f} seconds")
        print(f"Apriori Execution Time: {ap_duration:.4f} seconds")
        
        # Determine the faster algorithm
        faster_algorithm = min(("Brute Force", bf_duration), ("Apriori", ap_duration), key=lambda x: x[1])
        print(f"\nThe faster algorithm was: {faster_algorithm[0]} with a duration of {faster_algorithm[1]:.4f} seconds")
        
        # Compare results and display rules
        compare_algorithms((bf_itemsets, bf_rules), (ap_itemsets, ap_rules))
        show_rules(bf_rules, "Brute Force")
        show_rules(ap_rules, "Apriori")
        
        # Display item counts and support
        print("\nItem Counts:")
        item_count_dict = defaultdict(int)
        for transaction in transactions:
            for item in transaction:
                item_count_dict[item] += 1
        
        for item, count in item_count_dict.items():
            support_value = count / len(transactions)
            meets_threshold = "Meets" if support_value >= min_support else "Does not meet"
            print(f"{item}: Count = {count}, Support = {support_value:.2f} ({meets_threshold} support threshold)")

    except Exception as error:
        print(f"An error occurred: {str(error)}")
        print("Please verify your input data and try again.")
    
    # Ask user if they want to analyze another store
    print("\nWould you like to analyze another store? (y/n)")
    if input().lower() != 'y':
        break

print("Thank you for using the program!")


Welcome! Please select a store to analyze:
1. Amazon
2. Best Buy
3. Nike
4. K Mart
5. Target


Enter the number of the store you want to analyze (1-5):  3



Analyzing transactions for Nike:


Enter minimum support (1-100):  55
Enter minimum confidence (1-100):  70



Executing algorithms...

Brute Force Execution Time: 0.0012 seconds
Apriori Execution Time: 0.0098 seconds

The faster algorithm was: Brute Force with a duration of 0.0012 seconds

Results Comparison:
Brute Force: 8 frequent itemsets, 6 rules
Apriori: 8 frequent itemsets, 6 rules
Same number of frequent itemsets: True
Same number of rules: True

Brute Force Association Rules:
Rule 1: {'Socks'} -> {'Sweatshirts'} (Confidence: 0.92, Support: 0.60)
Rule 2: {'Sweatshirts'} -> {'Socks'} (Confidence: 0.92, Support: 0.60)
Rule 3: {'Sweatshirts'} -> {'Running Shoe'} (Confidence: 0.85, Support: 0.55)
Rule 4: {'Running Shoe'} -> {'Sweatshirts'} (Confidence: 0.79, Support: 0.55)
Rule 5: {'Socks'} -> {'Running Shoe'} (Confidence: 0.85, Support: 0.55)
Rule 6: {'Running Shoe'} -> {'Socks'} (Confidence: 0.79, Support: 0.55)

Apriori Association Rules:
Rule 1: {'Socks'} -> {'Running Shoe'} (Confidence: 0.85, Support: 0.55)
Rule 2: {'Running Shoe'} -> {'Socks'} (Confidence: 0.79, Support: 0.55)
Rule 3